In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys  
import os
import pickle
import threading
import multiprocessing as mp

In [2]:
import warnings
import Classes_definition as cp2p
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
warnings.filterwarnings('error')
%load_ext autoreload
%autoreload 2

In [69]:
def run_comm_decisions(data):
    '''
    Using a predetermined seed to initialize a pseudorandom number generator.
    Writes aggregated results of SC and P2P communities on a file.
    '''
    # Predefined data
    Data={'community_size':100, # number of households
     'timestep':0.25, # data resolution 0.25 = 15 min
     'seed':data['seed'], # For reproducibility
     'country_code':'DE', # other countries can be used (with other timesteps)
     'pv_penetration':50, # % of community with PV
     'batt_penetration':50, # % of households with PV that also have Battery
     'retail_price':0.28, # flat retail price
     'export_price':0.04, # flat FiT or export price
     'predet_bhv':data['bhv'], # different behaviours can be used (False, 'high',
                         # 'medium', 'low','cuts'). Gets the corresponding group of 
                         # the survey according to their answers on trading.
                         # The distribution is divided in three slots.
           
     'ILR':1, # inverter load ratio, default to 1
     'test_sc':False, # To test SC with P2P price structure
     'cut':'q0', # Quartiles of trading, similar to predet_bhv, but with higher
                 # resolution. The distribution is divided in ten slots.
     'PtC':False, # Possibility to use Production to consumption ratio, 
                   # instead of random allocation of PV capacities accross community
     'parallel':True # If multithreading is to be used, it deletes 
                      # attributes from the class to run quicker.
     }
    # Run SC community and save aggregated results$
    comP2PDict={}
    comNormal=cp2p.SCCommunity(Data) # initialize
    comNormal.runAll() # Run all methods
    filename='../Output/optimal_trading_all.csv'
    comNormal.out_hh.to_csv(filename, mode='a', header=False)
    comP2PDict=cp2p.P2PCommunity(Data) # initialize
    comP2PDict.getPrices(comNormal) 
    comP2PDict.getP2PCommunity()
    comP2PDict.endP2P()
    comP2PDict.out_hh.to_csv(filename, mode='a', header=False)
    return 

In [70]:
from itertools import product

def expand_grid(dct):
    rows = product(*dct.values())
    return pd.DataFrame.from_records(rows, columns=dct.keys())



In [71]:
dct={'seed':range(100),'bhv':['low','medium','high']}
data=expand_grid(dct)

In [72]:
data=[dict(data.loc[i,:]) for i in data.index]

In [75]:
#Run this for different cuts
mp.freeze_support()
pool=mp.Pool(processes=20)
pool.map(run_comm_decisions, data)
pool.close()
pool.join()

######################
######################
######################
######################
######################
######################
######################
######################
######################
######################
######################
######################
######################
######################
Getting demand and PV profiles
Getting demand and PV profiles
Getting demand and PV profiles
######################
Getting demand and PV profiles
######################
Getting demand and PV profiles
######################
Getting demand and PV profiles
Getting demand and PV profiles
######################
Getting demand and PV profiles
######################
######################
Getting demand and PV profiles
Getting demand and PV profiles
Getting demand and PV profiles
Getting demand and PV profiles
Getting demand and PV profiles
inside
inside
Getting demand and PV profiles
inside
inside
Getting demand and PV profiles
inside
Getting demand and PV profiles
inside
in

In [74]:
df2=pd.read_csv('../Output/sensitivity.csv')

In [37]:
df3=pd.read_csv('../Output/optimal_trading	.csv',names=df2.columns)

In [38]:
df3.head()

,Unnamed: 0,SCR,SSR,EFC,Demand_peak,Inj_peak,avg_dod,Total_load,TotalPV,bill,...,seed_x,hh,type,SCR_comm,SSR_comm,Bill_comm,cuts,Comm,predet_bhv,size_comm
0,0,62.456026,74.514382,186.604341,5.533404,3.625832,0.511245,5149.176417,6143.325519,296.869099,...,3,0,PV_batt,53.228753,47.095203,84203.405024,q0,SC,cuts,NaN
1,1,0.000000,0.000000,0.000000,6.797667,0.000000,0.000000,2663.394450,0.000000,745.750446,...,3,1,No,53.228753,47.095203,84203.405024,q0,SC,cuts,NaN
2,2,31.481269,92.474712,226.815131,4.808352,7.358761,0.621411,4287.327217,12593.817314,-216.167821,...,3,2,PV_batt,53.228753,47.095203,84203.405024,q0,SC,cuts,NaN
3,3,19.859074,46.356231,0.000000,4.778933,5.928663,0.000000,4342.485150,10136.487106,351.640940,...,3,3,PV,53.228753,47.095203,84203.405024,q0,SC,cuts,NaN
4,4,0.000000,0.000000,0.000000,6.539200,0.000000,0.000000,3238.714667,0.000000,906.840107,...,3,4,No,53.228753,47.095203,84203.405024,q0,SC,cuts,NaN


In [39]:
df3.columns

Index(['Unnamed: 0', 'SCR', 'SSR', 'EFC', 'Demand_peak', 'Inj_peak', 'avg_dod',
       'Total_load', 'TotalPV', 'bill', 'Imported', 'Exported', 'Batt_revenue',
       'Batt_penetration_x', 'PV_penetration_x', 'seed_x', 'hh', 'type',
       'SCR_comm', 'SSR_comm', 'Bill_comm', 'cuts', 'Comm', 'predet_bhv',
       'size_comm'],
      dtype='object')

In [65]:
(df3.groupby(['hh','seed_x','cuts','Comm']).sum().bill.unstack().diff(axis=1).unstack().unstack()<0).sum().SC.unstack().sum(axis=1)

cuts
q0      10
q90    742
dtype: int64